In [ ]:
import os, math, gc
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# -------- CONFIG --------
EVENTS = {
    "2008 Elections": "data/2008_elections",
    "2011 Occupy Wall Street": "data/2011_wallstreet",
    "2016 Elections": "data/2016_elections",
    "2017 Charlottesville Rally": "data/2017_rally",
    "2021 Capitol Riot": "data/2021_riot",
}

OUT_DIR = os.path.join("data", "thresholds_analysis")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV = os.path.join(OUT_DIR, "add_vs_mul_baseline_all_events.csv")
OUT_FIG = os.path.join(OUT_DIR, "fig_add_vs_mul_delta_auc_cv.png")

omega = 1.0
beta  = {"uil": 1.0, "csl": 1.0, "tdl": 1.0, "asl": 1.0}
LAYS  = ["uil","csl","tdl","asl"]

# -------- HELPERS (streaming, low-RAM) --------
def load_authors(path):
    with open(path, "r", encoding="utf-8") as f:
        return [ln.strip() for ln in f if ln.strip()]

def strength_stream_edges(path, authors_set):
    """Somma log1p(w) su u e v per ogni riga 'u;v;w'. Sparse dict."""
    s = defaultdict(float)
    if not os.path.isfile(path): return s
    with open(path, "r", encoding="utf-8") as fin:
        for line in fin:
            parts = line.rstrip("\n").split(";")
            if len(parts) != 3: continue
            u, v, ws = parts
            if (u not in authors_set) or (v not in authors_set): continue
            try:
                w = float(ws)
            except ValueError:
                continue
            inc = math.log1p(w)
            s[u] += inc; s[v] += inc
    return s

def effective_phi_per_layer(str_by_layer, authors, omega=1.0):
    """Ritorna matrice N x 4 delle effective influence per layer."""
    n = len(authors)
    Ef = np.zeros((n, len(LAYS)), dtype=np.float64)
    totals = np.zeros(n, dtype=np.float64)
    # pre-somma totals
    for j,l in enumerate(LAYS):
        for i,a in enumerate(authors):
            totals[i] += str_by_layer[l].get(a, 0.0)
    denom = max(n-1, 1)
    for j,l in enumerate(LAYS):
        for i,a in enumerate(authors):
            phi = str_by_layer[l].get(a, 0.0)
            eff = (phi + omega * (totals[i] - phi)) / denom
            Ef[i,j] = eff
    return Ef

def fit_alpha_ci_auc_on_score(s, y):
    """Logit(y ~ s). Ritorna alpha, lo, hi, auc_in."""
    s = (s - np.mean(s)) / (np.std(s) if np.std(s)>0 else 1.0)
    alpha = np.nan; lo = np.nan; hi = np.nan; auc = np.nan
    try:
        import statsmodels.api as sm
        X = sm.add_constant(s)
        res = sm.Logit(y, X).fit(disp=0)
        alpha = float(res.params[1])
        ci = res.conf_int(alpha=0.05)
        lo, hi = float(ci.iloc[1,0]), float(ci.iloc[1,1])
        p = res.predict(X)
        auc = roc_auc_score(y, p) if len(np.unique(y))==2 else np.nan
        return alpha, lo, hi, auc
    except Exception:
        lr = LogisticRegression(solver="liblinear")
        s2 = s.reshape(-1,1)
        lr.fit(s2, y)
        p = lr.predict_proba(s2)[:,1]
        auc = roc_auc_score(y, p)
        alpha = float(lr.coef_.ravel()[0]); lo = hi = np.nan
        return alpha, lo, hi, auc

def cv_auc_for_score(s, y, n_splits=5, seed=42):
    """5-fold CV AUC: in ogni fold fit logit(y~s) sul train e predici sul test."""
    s = (s - np.mean(s)) / (np.std(s) if np.std(s)>0 else 1.0)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    aucs = []
    for tr, te in skf.split(np.zeros_like(y), y):
        lr = LogisticRegression(solver="liblinear")
        lr.fit(s[tr].reshape(-1,1), y[tr])
        p = lr.predict_proba(s[te].reshape(-1,1))[:,1]
        aucs.append(roc_auc_score(y[te], p))
    return float(np.mean(aucs)), float(np.std(aucs))

def topk_set(score, authors, frac=0.01):
    k = max(1, int(math.ceil(frac*len(authors))))
    idx = np.argsort(-score, kind="stable")[:k]
    return set([authors[i] for i in idx])

def jaccard(a,b):
    if not a and not b: return 1.0
    u = len(a|b); return len(a&b)/u if u>0 else 0.0

# -------- MAIN LOOP --------
rows = []
for ev_name, ev_folder in EVENTS.items():
    AUTHORS_FILE = os.path.join(ev_folder, "network", "authors.txt")
    LABELS_FILE  = os.path.join(ev_folder, "CRC_radicalization_analysis.csv")
    UIL_FILE     = os.path.join(ev_folder, "network", "uil", "edges.txt")
    CSL_FILE     = os.path.join(ev_folder, "network", "csl", "edges.txt")   # baseline τ_c already applied
    TDL_FILE     = os.path.join(ev_folder, "network", "tdl", "edges.txt")
    ASL_FILE     = os.path.join(ev_folder, "network", "asl", "edges.txt")   # baseline τ_a already applied

    if not (os.path.isfile(AUTHORS_FILE) and os.path.isfile(LABELS_FILE)):
        print(f"[skip] Missing authors/labels for {ev_name}")
        continue

    # Autori + etichette (manteniamo l'ordine del file authors.txt)
    authors = load_authors(AUTHORS_FILE)
    labs = pd.read_csv(LABELS_FILE, usecols=["author","radical"]).astype({"author":str})
    lab_set = set(labs["author"])
    authors = [a for a in authors if a in lab_set]
    y = labs.set_index("author").loc[authors, "radical"].astype("int8").values
    if len(np.unique(y)) < 2:
        print(f"[skip] Only one class in labels for {ev_name}")
        continue
    authors_set = set(authors)

    # Strength per layer (streaming)
    str_uil = strength_stream_edges(UIL_FILE, authors_set)
    str_csl = strength_stream_edges(CSL_FILE, authors_set)  # file già filtrato a baseline
    str_tdl = strength_stream_edges(TDL_FILE, authors_set)
    str_asl = strength_stream_edges(ASL_FILE, authors_set)

    # Effective influence matrix N x 4
    Ef = effective_phi_per_layer(
        {"uil":str_uil,"csl":str_csl,"tdl":str_tdl,"asl":str_asl},
        authors, omega=omega
    )
    # Z-score per colonna per stabilità e per Z-sum
    Ef_mean = Ef.mean(axis=0, keepdims=True)
    Ef_std  = Ef.std(axis=0, keepdims=True); Ef_std[Ef_std==0] = 1.0
    Ef_z = (Ef - Ef_mean) / Ef_std

    # ---- Indici ----
    betavec = np.array([beta["uil"], beta["csl"], beta["tdl"], beta["asl"]], dtype=np.float64)
    # CRC× (multiplicative)
    crc_mul = np.prod(1.0 + Ef * betavec, axis=1)
    # CRC+ (additive)
    crc_add = np.sum(Ef * betavec, axis=1)
    # Z-sum (media delle colonne z-scored)
    z_sum   = Ef_z.mean(axis=1)

    # Linear (learned): predittore lineare (senza intercetta) da logit su Ef_z
    s_lin = None
    try:
        import statsmodels.api as sm
        X4 = sm.add_constant(Ef_z)
        res4 = sm.Logit(y, X4).fit(disp=0)
        theta = np.asarray(res4.params[1:], dtype=float)  # 4 pesi (skip intercept)
        s_lin = (Ef_z @ theta).astype(float)
    except Exception:
        lr4 = LogisticRegression(solver="liblinear")
        lr4.fit(Ef_z, y)
        theta = lr4.coef_.ravel().astype(float)
        s_lin = (Ef_z @ theta).astype(float)

    # ---- Metriche per ogni indice ----
    def eval_index(idx_name, s):
        alpha, lo, hi, auc_in = fit_alpha_ci_auc_on_score(s, y)
        auc_cv_mean, auc_cv_sd = cv_auc_for_score(s, y, n_splits=5, seed=42)
        return {
            "Event": ev_name, "Index": idx_name,
            "alpha": alpha, "alpha_lo": lo, "alpha_hi": hi,
            "AUC_in": auc_in, "AUC_cv_mean": auc_cv_mean, "AUC_cv_sd": auc_cv_sd
        }

    rows.append(eval_index("CRC× (multiplicative)", crc_mul))
    rows.append(eval_index("CRC+ (additive)",      crc_add))
    rows.append(eval_index("Z-sum",                z_sum))
    rows.append(eval_index("Linear (learned)",     s_lin))

    # ---- Ranking overlap (top-1%) vs CRC× ----
    base_top1 = topk_set(crc_mul, authors, frac=0.01)
    for name, s in [("CRC+ (additive)", crc_add), ("Z-sum", z_sum), ("Linear (learned)", s_lin)]:
        s_top1 = topk_set(s, authors, frac=0.01)
        rows.append({
            "Event": ev_name, "Index": f"Jaccard vs CRC× — {name}",
            "alpha": np.nan, "alpha_lo": np.nan, "alpha_hi": np.nan,
            "AUC_in": jaccard(base_top1, s_top1),
            "AUC_cv_mean": np.nan, "AUC_cv_sd": np.nan
        })

    # free per-event
    del str_uil, str_csl, str_tdl, str_asl, Ef, Ef_z, crc_mul, crc_add, z_sum, s_lin
    gc.collect()

# -------- Save & quick viz --------
df = pd.DataFrame(rows)
df.to_csv(OUT_CSV, index=False)
print(f"Saved → {OUT_CSV}")
display(df)

# Optional: barplot ΔAUC_cv (index - CRC×) per evento
try:
    import matplotlib.pyplot as plt
    pivot = df[df["Index"].isin(["CRC× (multiplicative)","CRC+ (additive)","Z-sum","Linear (learned)"])].pivot_table(
        index=["Event","Index"], values="AUC_cv_mean", aggfunc="first"
    ).reset_index()
    base = pivot[pivot["Index"]=="CRC× (multiplicative)"][["Event","AUC_cv_mean"]].rename(columns={"AUC_cv_mean":"base"})
    merged = pivot.merge(base, on="Event")
    merged["DeltaAUC"] = merged["AUC_cv_mean"] - merged["base"]

    fig, ax = plt.subplots(figsize=(8.5, 4.2))
    for ev in merged["Event"].unique():
        sub = merged[merged["Event"]==ev]
        ax.plot(sub["Index"], sub["DeltaAUC"], marker="o", label=ev)
    ax.axhline(0.0, linestyle="--", linewidth=1, color="0.3")
    ax.set_ylabel("ΔAUC (5-fold CV) vs CRC×")
    ax.set_xlabel("Index")
    ax.set_xticklabels(["CRC×","CRC+","Z-sum","Linear"], rotation=0)
    ax.legend(frameon=False, ncol=2)
    fig.tight_layout()
    fig.savefig(OUT_FIG, bbox_inches="tight", dpi=150)
    plt.close(fig)
    print(f"Saved figure → {OUT_FIG}")
except Exception as e:
    print("Plot skipped:", e)

In [ ]:
import os, math, gc
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.metrics import average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# -------- CONFIG --------
EVENTS = {
    "2008 Elections": "data/2008_elections",
    "2011 Occupy Wall Street": "data/2011_wallstreet",
    "2016 Elections": "data/2016_elections",
    "2017 Charlottesville Rally": "data/2017_rally",
    "2021 Capitol Riot": "data/2021_riot",
}

OUT_DIR = os.path.join("data", "thresholds_analysis")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV = os.path.join(OUT_DIR, "add_vs_mul_topk_summary.csv")
OUT_TEX = os.path.join(OUT_DIR, "add_vs_mul_topk_summary.tex")

# Top-k fractions to assess tail behavior
TOPK_FRACS = [0.005, 0.01, 0.05]  # 0.5%, 1%, 5%
INDEX_NAMES = ["CRC×", "CRC+", "Z-sum", "Linear"]

# -------- HELPERS (streaming, low-RAM) --------
def load_authors(path):
    with open(path, "r", encoding="utf-8") as f:
        return [ln.strip() for ln in f if ln.strip()]

def strength_stream_edges(path, authors_set):
    """Somma log1p(w) su u e v per ogni riga 'u;v;w' in streaming. Ritorna dict sparse."""
    s = defaultdict(float)
    if not os.path.isfile(path): 
        return s
    with open(path, "r", encoding="utf-8") as fin:
        for line in fin:
            parts = line.rstrip("\n").split(";")
            if len(parts) != 3: 
                continue
            u, v, ws = parts
            if (u not in authors_set) or (v not in authors_set): 
                continue
            try:
                w = float(ws)
            except ValueError:
                continue
            inc = math.log1p(w)
            s[u] += inc; s[v] += inc
    return s

def effective_phi_matrix(str_by_layer, authors, omega=1.0, layer_order=("uil","csl","tdl","asl")):
    """N x L matrix of normalized effective influence per layer."""
    n = len(authors)
    L = len(layer_order)
    Ef = np.zeros((n, L), dtype=np.float64)
    totals = np.zeros(n, dtype=np.float64)
    for j,l in enumerate(layer_order):
        for i,a in enumerate(authors):
            totals[i] += str_by_layer[l].get(a, 0.0)
    denom = max(n-1, 1)
    for j,l in enumerate(layer_order):
        for i,a in enumerate(authors):
            phi = str_by_layer[l].get(a, 0.0)
            Ef[i,j] = (phi + omega * (totals[i] - phi)) / denom
    return Ef

def build_indices(Ef, y):
    """Return dict of indices on Ef: CRC×, CRC+, Z-sum, Linear (learned)."""
    # z-score per colonna (per Z-sum e linear)
    Ef_mean = Ef.mean(axis=0, keepdims=True)
    Ef_std  = Ef.std(axis=0, keepdims=True); Ef_std[Ef_std==0] = 1.0
    Ef_z = (Ef - Ef_mean) / Ef_std

    # CRC× (multiplicative) and CRC+ (additive)
    crc_mul = np.prod(1.0 + Ef, axis=1)           # beta=1 su tutti i layer
    crc_add = np.sum(Ef, axis=1)

    # Z-sum (media dei z-scores)
    z_sum   = Ef_z.mean(axis=1)

    # Linear (learned) via logit su Ef_z
    try:
        import statsmodels.api as sm
        X = sm.add_constant(Ef_z)
        res = sm.Logit(y, X).fit(disp=0)
        theta = np.asarray(res.params[1:], dtype=float)   # skip intercept
        s_lin = (Ef_z @ theta).astype(float)
    except Exception:
        lr = LogisticRegression(solver="liblinear")
        lr.fit(Ef_z, y)
        theta = lr.coef_.ravel().astype(float)
        s_lin = (Ef_z @ theta).astype(float)

    return {
        "CRC×": crc_mul,
        "CRC+": crc_add,
        "Z-sum": z_sum,
        "Linear": s_lin,
        "Ef_z": Ef_z  # usato solo per la metrica di non-compensazione
    }

def precision_at_k(score, y, frac):
    n = len(y)
    k = max(1, int(math.ceil(frac * n)))
    idx = np.argsort(-score, kind="stable")[:k]
    return float(y[idx].mean()), k

def noncomp_fraction_at_k(Ef_z, score, frac, thresh=0.0):
    """Quota di utenti nel top-k con z>thresh in *tutti* i layer."""
    n = Ef_z.shape[0]
    k = max(1, int(math.ceil(frac * n)))
    idx = np.argsort(-score, kind="stable")[:k]
    Ztop = Ef_z[idx, :]
    ok_all = np.all(Ztop > thresh, axis=1)
    return float(ok_all.mean())

def prevalence(y):
    return float(np.mean(y))

# -------- MAIN --------
rows = []
for ev_name, ev_folder in EVENTS.items():
    AUTHORS_FILE = os.path.join(ev_folder, "network", "authors.txt")
    LABELS_FILE  = os.path.join(ev_folder, "CRC_radicalization_analysis.csv")
    UIL_FILE     = os.path.join(ev_folder, "network", "uil", "edges.txt")
    CSL_FILE     = os.path.join(ev_folder, "network", "csl", "edges.txt")   # baseline τ_c già applicato
    TDL_FILE     = os.path.join(ev_folder, "network", "tdl", "edges.txt")
    ASL_FILE     = os.path.join(ev_folder, "network", "asl", "edges.txt")   # baseline τ_a già applicato

    if not (os.path.isfile(AUTHORS_FILE) and os.path.isfile(LABELS_FILE)):
        print(f"[skip] Missing authors/labels for {ev_name}")
        continue

    # Autori + etichette nell'ordine di authors.txt
    authors = load_authors(AUTHORS_FILE)
    labs = pd.read_csv(LABELS_FILE, usecols=["author","radical"]).astype({"author":str})
    lab_set = set(labs["author"])
    authors = [a for a in authors if a in lab_set]
    y = labs.set_index("author").loc[authors, "radical"].astype("int8").values
    if len(np.unique(y)) < 2:
        print(f"[skip] Only one class in labels for {ev_name}")
        continue
    authors_set = set(authors)

    # Strength per layer (streaming)
    str_uil = strength_stream_edges(UIL_FILE, authors_set)
    str_csl = strength_stream_edges(CSL_FILE, authors_set)
    str_tdl = strength_stream_edges(TDL_FILE, authors_set)
    str_asl = strength_stream_edges(ASL_FILE, authors_set)

    # Matrice N x 4 delle effective influences
    Ef = effective_phi_matrix(
        {"uil":str_uil,"csl":str_csl,"tdl":str_tdl,"asl":str_asl},
        authors, omega=1.0
    )

    # Indici
    idx = build_indices(Ef, y)
    p_base = prevalence(y)

    # PR-AUC (Average Precision) per indice
    pr_auc = {}
    for name in INDEX_NAMES:
        s = idx[name]
        # Normalizza per stabilità
        s_std = (s - s.mean()) / (s.std() if s.std()>0 else 1.0)
        pr_auc[name] = average_precision_score(y, s_std)

    # Metriche top-k e non-compensazione
    for frac in TOPK_FRACS:
        for name in INDEX_NAMES:
            s = idx[name]
            prec_k, k = precision_at_k(s, y, frac)
            lift_k = prec_k / p_base if p_base > 0 else np.nan
            noncomp_k = noncomp_fraction_at_k(idx["Ef_z"], s, frac, thresh=0.0)
            rows.append({
                "Event": ev_name,
                "Index": name,
                "k_frac": frac,
                "k_abs": k,
                "Prevalence": p_base,
                "Precision@k": prec_k,
                "Lift@k": lift_k,
                "NonComp@AllLayers@k": noncomp_k,
                "PR-AUC": pr_auc[name],
            })

    # cleanup
    del str_uil, str_csl, str_tdl, str_asl, Ef, idx
    gc.collect()

# Save
df = pd.DataFrame(rows)
df.to_csv(OUT_CSV, index=False)
print(f"Saved → {OUT_CSV}")

# Make a compact LaTeX table at k=1% (space-friendly)
sub = df[df["k_frac"]==0.01].copy()
# Wide pivot: one row per event, columns = (Index × metric)
wide = sub.pivot_table(index="Event", columns="Index", values=["PR-AUC","Precision@k","Lift@k","NonComp@AllLayers@k"])
# Order columns nicely
wide = wide.reindex(columns=pd.MultiIndex.from_product(
    [["PR-AUC","Precision@k","Lift@k","NonComp@AllLayers@k"], INDEX_NAMES]
))
# Format
tex = wide.round(3).to_latex(na_rep="", escape=False, multirow=True, multicolumn=True, column_format="lcccccccccccc")
with open(OUT_TEX, "w") as f:
    f.write(tex)
print(f"Saved → {OUT_TEX}")

In [ ]:
# === Multiplicative vs Additive: exclusive enrichment + CV interactions + top-k co-activation ===
# Outputs:
#   data/thresholds_analysis/synergy_exclusive_enrichment.csv / .tex
#   data/thresholds_analysis/add_vs_int_cv_deltas.csv / .tex
#   data/thresholds_analysis/topk_minz_compare.csv / .tex

import os, math, itertools, gc
import numpy as np
import pandas as pd

from collections import defaultdict
from scipy.stats import fisher_exact, wilcoxon
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import log_loss, roc_auc_score

# -------- CONFIG --------
EVENTS = {
    "2008 Elections": "data/2008_elections",
    "2011 Occupy Wall Street": "data/2011_wallstreet",
    "2016 Elections": "data/2016_elections",
    "2017 Charlottesville Rally": "data/2017_rally",
    "2021 Capitol Riot": "data/2021_riot",
}
OUT_DIR = os.path.join("data", "thresholds_analysis"); os.makedirs(OUT_DIR, exist_ok=True)

ENR_CSV = os.path.join(OUT_DIR, "synergy_exclusive_enrichment.csv")
ENR_TEX = os.path.join(OUT_DIR, "synergy_exclusive_enrichment.tex")
CV_CSV  = os.path.join(OUT_DIR, "add_vs_int_cv_deltas.csv")
CV_TEX  = os.path.join(OUT_DIR, "add_vs_int_cv_deltas.tex")
TOPK_CSV= os.path.join(OUT_DIR, "topk_minz_compare.csv")
TOPK_TEX= os.path.join(OUT_DIR, "topk_minz_compare.tex")

LAYS = ["uil","csl","tdl","asl"]
QS   = [0.7, 0.8, 0.9]   # quantili candidati
LOWQ = 0.5               # "basso" = ≤ mediana per gli altri layer
N_MIN = 30               # minimo elementi per gruppo
TOPK_FRAC = 0.01         # top-1%

# -------- IO helpers --------
def load_authors(path):
    with open(path, "r", encoding="utf-8") as f:
        return [ln.strip() for ln in f if ln.strip()]

def strength_stream_edges(path, authors_set):
    s = defaultdict(float)
    if not os.path.isfile(path): return s
    with open(path, "r", encoding="utf-8") as fin:
        for line in fin:
            parts = line.rstrip("\n").split(";")
            if len(parts)!=3: continue
            u,v,ws = parts
            if (u not in authors_set) or (v not in authors_set): continue
            try: w = float(ws)
            except: continue
            inc = math.log1p(w)
            s[u] += inc; s[v] += inc
    return s

def effective_phi_matrix(str_by_layer, authors, omega=1.0, layer_order=("uil","csl","tdl","asl")):
    n = len(authors); L = len(layer_order)
    Ef = np.zeros((n,L), dtype=np.float64)
    totals = np.zeros(n, dtype=np.float64)
    for j,l in enumerate(layer_order):
        for i,a in enumerate(authors):
            totals[i] += str_by_layer[l].get(a,0.0)
    denom = max(n-1,1)
    for j,l in enumerate(layer_order):
        for i,a in enumerate(authors):
            phi = str_by_layer[l].get(a,0.0)
            Ef[i,j] = (phi + omega*(totals[i]-phi))/denom
    return Ef

def zscore_cols(X):
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, keepdims=True); sd[sd==0]=1.0
    return (X-mu)/sd

# -------- exclusive enrichment --------
def exclusive_masks(Ef_z, q, lowq=0.5):
    """Restituisce:
       - all_high_excl: z >= q-quantile su TUTTI i layer
       - single_high_excl: z >= q-quantile su UNO e z <= lowq-quantile su TUTTI gli altri
    """
    n,L = Ef_z.shape
    thr_hi = np.quantile(Ef_z, q, axis=0)
    thr_lo = np.quantile(Ef_z, lowq, axis=0)
    H = (Ef_z >= thr_hi)   # N x L
    Lw = (Ef_z <= thr_lo)  # N x L
    all_high = H.all(axis=1)
    single_high = np.zeros(n, dtype=bool)
    for j in range(L):
        cond = H[:,j] & np.all(Lw[:,np.arange(L)!=j], axis=1)
        single_high |= cond
    return all_high, single_high, H.sum(axis=1)

def risk_ratio(all_high, single_high, y, eps=1e-6):
    A_pos = int(((y==1)&all_high).sum()); A_neg = int(((y==0)&all_high).sum())
    B_pos = int(((y==1)&single_high).sum()); B_neg = int(((y==0)&single_high).sum())
    if (A_pos+A_neg)<1 or (B_pos+B_neg)<1:
        return np.nan, np.nan, A_pos, A_neg, B_pos, B_neg
    risk_A = (A_pos+eps)/(A_pos+A_neg+eps)
    risk_B = (B_pos+eps)/(B_pos+B_neg+eps)
    RR = risk_A/risk_B
    _, p = fisher_exact([[A_pos,A_neg],[B_pos,B_neg]], alternative="greater")
    return RR, float(p), A_pos, A_neg, B_pos, B_neg

# -------- CV additive vs interactions (L2) --------
from sklearn.model_selection import StratifiedKFold
def inter_feat(Z):
    cols=[Z]; L=Z.shape[1]
    for i,j in itertools.combinations(range(L),2):
        cols.append((Z[:,i]*Z[:,j])[:,None])
    return np.hstack(cols)

def cv_add_vs_int(Ef_z, y, seed=42):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    ll_add, ll_int, auc_add, auc_int = [], [], [], []
    for tr,te in skf.split(Ef_z, y):
        ss_add = StandardScaler().fit(Ef_z[tr])
        Xtr_add = ss_add.transform(Ef_z[tr]); Xte_add = ss_add.transform(Ef_z[te])
        ss_int = StandardScaler().fit(inter_feat(Ef_z[tr]))
        Xtr_int = ss_int.transform(inter_feat(Ef_z[tr])); Xte_int = ss_int.transform(inter_feat(Ef_z[te]))
        clf_add = LogisticRegressionCV(Cs=10, cv=3, scoring='neg_log_loss', solver='lbfgs', penalty='l2', max_iter=2000)
        clf_int = LogisticRegressionCV(Cs=10, cv=3, scoring='neg_log_loss', solver='lbfgs', penalty='l2', max_iter=2000)
        clf_add.fit(Xtr_add, y[tr]); clf_int.fit(Xtr_int, y[tr])
        p_add = clf_add.predict_proba(Xte_add)[:,1]; p_int = clf_int.predict_proba(Xte_int)[:,1]
        ll_add.append(log_loss(y[te], p_add)); ll_int.append(log_loss(y[te], p_int))
        auc_add.append(roc_auc_score(y[te], p_add)); auc_int.append(roc_auc_score(y[te], p_int))
    return np.array(ll_add), np.array(ll_int), np.array(auc_add), np.array(auc_int)

# -------- top-k co-activation (min-z) --------
def topk_minz(Ef_z, scores, frac=0.01):
    n = Ef_z.shape[0]; k = max(1, int(math.ceil(frac*n)))
    out={}
    for name, s in scores.items():
        idx = np.argsort(-s, kind="stable")[:k]
        out[name] = float(np.mean(Ef_z[idx,:].min(axis=1)))
    return out

# -------- MAIN --------
rows_enr = []
rows_cv  = []
rows_topk= []

for ev_name, ev_folder in EVENTS.items():
    AUTHORS = os.path.join(ev_folder, "network", "authors.txt")
    LABELS  = os.path.join(ev_folder, "CRC_radicalization_analysis.csv")
    UIL     = os.path.join(ev_folder, "network", "uil", "edges.txt")
    CSL     = os.path.join(ev_folder, "network", "csl", "edges.txt")
    TDL     = os.path.join(ev_folder, "network", "tdl", "edges.txt")
    ASL     = os.path.join(ev_folder, "network", "asl", "edges.txt")
    if not (os.path.isfile(AUTHORS) and os.path.isfile(LABELS)): 
        print(f"[skip] {ev_name}"); continue

    authors = load_authors(AUTHORS)
    labs = pd.read_csv(LABELS, usecols=["author","radical"]).astype({"author":str})
    lab_set = set(labs["author"])
    authors = [a for a in authors if a in lab_set]
    y = labs.set_index("author").loc[authors,"radical"].astype("int8").values
    if len(np.unique(y))<2: 
        print(f"[skip one-class] {ev_name}"); continue
    Aset = set(authors)

    str_uil = strength_stream_edges(UIL, Aset)
    str_csl = strength_stream_edges(CSL, Aset)
    str_tdl = strength_stream_edges(TDL, Aset)
    str_asl = strength_stream_edges(ASL, Aset)

    Ef = effective_phi_matrix({"uil":str_uil,"csl":str_csl,"tdl":str_tdl,"asl":str_asl}, authors, omega=1.0)
    Ef_z = zscore_cols(Ef)

    # --- (1) exclusive enrichment ---
    best = None
    for q in QS:
        ah, sh, _ = exclusive_masks(Ef_z, q=q, lowq=LOWQ)
        if ah.sum()>=N_MIN and sh.sum()>=N_MIN:
            RR, p, Apos, Aneg, Bpos, Bneg = risk_ratio(ah, sh, y)
            best = (q, ah.sum(), sh.sum(), RR, p, Apos, Aneg, Bpos, Bneg)
            break
    rows_enr.append({
        "Event": ev_name,
        "q_used": best[0] if best else np.nan,
        "all_high_n": int(best[1]) if best else 0,
        "single_high_n": int(best[2]) if best else 0,
        "RiskRatio_all/single": float(best[3]) if best else np.nan,
        "Fisher_p": float(best[4]) if best else np.nan,
        "A_pos": int(best[5]) if best else 0, "A_neg": int(best[6]) if best else 0,
        "B_pos": int(best[7]) if best else 0, "B_neg": int(best[8]) if best else 0
    })

    # --- (2) CV additive vs interactions ---
    ll_add, ll_int, auc_add, auc_int = cv_add_vs_int(Ef_z, y, seed=42)
    dLL  = ll_int - ll_add   # <0 = interactions better
    dAUC = auc_int - auc_add # >0 = interactions better
    # Wilcoxon accoppiato sulle 5 fold (se tutte finite)
    try:
        w_p_ll = wilcoxon(ll_add, ll_int, alternative="greater").pvalue  # H1: add > int (int migliore)
    except Exception:
        w_p_ll = np.nan
    try:
        w_p_auc = wilcoxon(auc_int, auc_add, alternative="greater").pvalue  # H1: int > add
    except Exception:
        w_p_auc = np.nan
    rows_cv.append({
        "Event": ev_name,
        "mean_dLogLoss(int-add)": float(np.mean(dLL)),
        "mean_dAUC(int-add)": float(np.mean(dAUC)),
        "wilcoxon_p_logloss(add>int)": float(w_p_ll),
        "wilcoxon_p_auc(int>add)": float(w_p_auc),
        "folds_dLogLoss": ";".join([f"{x:.4f}" for x in dLL]),
        "folds_dAUC": ";".join([f"{x:.4f}" for x in dAUC])
    })

    # --- (3) top-1% co-activation (min z across layers) ---
    crc_mul = np.prod(1.0 + Ef, axis=1)
    crc_add = np.sum(Ef, axis=1)
    out = topk_minz(Ef_z, {"CRC×": crc_mul, "CRC+": crc_add}, frac=TOPK_FRAC)
    rows_topk.append({
        "Event": ev_name,
        "k_frac": TOPK_FRAC,
        "mean_minZ@topk (CRC×)": out["CRC×"],
        "mean_minZ@topk (CRC+)": out["CRC+"],
        "Δ(minZ) CRC×-CRC+": out["CRC×"] - out["CRC+"]
    })

# ----- SAVE & LaTeX -----
enr = pd.DataFrame(rows_enr); enr.to_csv(ENR_CSV, index=False)
cv  = pd.DataFrame(rows_cv);  cv.to_csv(CV_CSV, index=False)
topk= pd.DataFrame(rows_topk);topk.to_csv(TOPK_CSV, index=False)
print("Saved ->", ENR_CSV, CV_CSV, TOPK_CSV)

def to_tex(df, fname, caption, label, cols, rnd):
    d = df.copy()
    for c, r in rnd.items():
        if c in d.columns:
            d[c] = d[c].map(lambda x: "" if pd.isna(x) else f"{x:.{r}f}")
    d = d[cols]
    tex = d.to_latex(index=False, escape=False, caption=caption, label=label)
    print(f"{fname} {text}")
    with open(fname, "w") as f:
        f.write(tex)
    print("Saved ->", fname)

to_tex(
    enr, ENR_TEX,
    "Exclusive co-activation enrichment: all-high (top-$q$ on all layers) vs single-high (top-$q$ on exactly one layer and ≤ median on the others).",
    "tab:enrichment_exclusive",
    cols=["Event","q_used","all_high_n","single_high_n","RiskRatio_all/single","Fisher_p"],
    rnd={"RiskRatio_all/single":2,"Fisher_p":3}
)

to_tex(
    cv, CV_TEX,
    "Cross-validated comparison of additive vs additive+interactions (L2 logistic): negative mean $\\Delta$log-loss (int$-$add) favors interactions; positive mean $\\Delta$AUC favors interactions.",
    "tab:add_vs_int_cv",
    cols=["Event","mean_dLogLoss(int-add)","wilcoxon_p_logloss(add>int)","mean_dAUC(int-add)","wilcoxon_p_auc(int>add)"],
    rnd={"mean_dLogLoss(int-add)":4,"mean_dAUC(int-add)":4,"wilcoxon_p_logloss(add>int)":3,"wilcoxon_p_auc(int>add)":3}
)

to_tex(
    topk, TOPK_TEX,
    "Top-1\\% co-activation: mean of the minimum $z$ across layers for users selected by CRC$\\times$ vs CRC$+$ (higher is more jointly high).",
    "tab:topk_minz",
    cols=["Event","k_frac","mean_minZ@topk (CRC×)","mean_minZ@topk (CRC+)","Δ(minZ) CRC×-CRC+"],
    rnd={"k_frac":3,"mean_minZ@topk (CRC×)":3,"mean_minZ@topk (CRC+)":3,"Δ(minZ) CRC×-CRC+":3}
)